In [1]:
import os
import pandas as pd

directory = "/Users/Me/Documents/Bee /Bee project new/Bee project from 09.2022/Python"
log_folder = "Raw data H1H2H3H4"
processed_folder = "H1H2H3H4 1step process 110322"
tag_id_file = 'tagnumeros_09.2022.csv'

# # Replace with file path
# directory = '/Users/Me/Documents/Bee /Bee project new/Bee project from 09.2022/Python'
# log_folder="Raw data H3H4"
# #create a folder called this in the directory
# processed_folder='H3H4 Processed'
# tag_id_file='tagnumeros_09.2022.csv'

# List with all start time | all end times
                    # All trips less than 1 minute | number of trips less than 1 minute
                    # All trips 1-3 minutes | number of trips 1-3 minutes
                    # All trips 3-6 minutes | number of trips 3-6 minutes
                    # All trips more than 6 minutes | number of trips more than 6 minutes

# headers=["UID", "Day", "Group", "Trip Duration", "Rest Duration", "Start Time", 
# "End Time", "Trip Start and End Times", "All Start Times",  "All End Times", "Sub 1 Minute"]

activity = [[dict() for days in range(31)] for months in range(12)]
# latest could be December 31
start_month=11
start_day=30
# earliest could be January 1
end_month=0
end_day=0
# file_count = [[0 for days in range(31)] for months in range(12)]

# for filename in os.listdir(directory+'/'+log_folder):
#     if filename.endswith('.csv'):
#         #print(filename)
#         file_length = len(filename)
#         if filename[file_length-6:file_length-5] == ' ':
#             subtract = 16
#         elif filename[file_length-8:file_length-4] == 'copy':
#             subtract=19
#         else:
#             subtract = 14
#         month = int(filename[file_length-(subtract+5):file_length-(subtract+3)])-1
#         day = int(filename[file_length-(subtract+2):file_length-subtract])-1
#         file_count[month][day]+=1

def seconds_to_hours(original_num):
    seconds=original_num*3600
    hours = int(seconds/3600)
    seconds -= hours*3600
    minutes = int(seconds/60)
    seconds -= minutes*60
    seconds=int(seconds)

    hours_str=str(hours)
    if(len(hours_str)==1):
        hours_str='0'+hours_str
    minutes_str=str(minutes)
    if(len(minutes_str)==1):
        minutes_str='0'+minutes_str
    seconds_str=str(seconds)
    if(len(seconds_str)==1):
        seconds_str='0'+seconds_str

    return hours_str+":"+minutes_str+":"+seconds_str

treatment_ids_csv = pd.read_csv(os.path.join(directory, tag_id_file), engine='python', header=None)
treatment_ids={}
for i in range(len(treatment_ids_csv[0])):
    treatment_ids[treatment_ids_csv.loc[i][1]]=treatment_ids_csv.loc[i][0]

for filename in os.listdir(directory+'/'+log_folder):
    if filename.endswith('.csv'):
        #print(filename)
        pathname = os.path.join(directory, log_folder, filename)

        file_length = len(filename)
        if filename[file_length-6:file_length-5] == ' ':
            subtract = 16
        elif filename[file_length-8:file_length-4] == 'copy':
            subtract=19
        else:
            subtract = 14
        month = int(filename[file_length-(subtract+5):file_length-(subtract+3)])-1
        day = int(filename[file_length-(subtract+2):file_length-subtract])-1

        if month<start_month:
            start_month=month
            start_day=day
        elif month==start_month and day<start_day:
            start_day=day

        if month>end_month:
            end_month=month
            end_day=day
        elif month==end_month and day>end_day:
            end_day=day

        # file_count[month][day]-=1

        try:
            df = pd.read_csv(pathname, engine='python',
                            sep="[;,. ]", skiprows=24)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame()
            
        if not df.empty:
            df.dropna(axis=1, how='all', inplace=True)

            temp_columns = df.columns.tolist()
            df.loc[-1] = temp_columns
            df.index = df.index+1
            df = df.sort_index()

            time_found=False
            for i in range(len(temp_columns)):
                column = temp_columns[i]
                col_length = len(column)

                #temp_length=len(column)
                if column[2:3] == ":" and not time_found:
                    temp_columns[i] = "Timestamp"
                    time_found=True
                elif column[:4] == "25SQ":
                    temp_columns[i] = "UID"
                elif column == "1" or column == "2":
                    temp_columns[i] = "Ant. Number"

            df.columns = temp_columns

            for column in df.columns:
                if column != "Timestamp" and column != "UID" and column != "Ant. Number":
                    del(df[column])
            
            ids = len(df["UID"])
            keys=set()
            

            for i in range(ids):
                key = df["UID"][i]
                if not key in keys:
                    keys.add(key)

                    #day number
                    # Group ID
                    # List with durations of trips in sequential order | 
                    # List with durations of time between trips in sequential order | 
                    # Start time of first trip | end time of last trip
                    # List with start and end times of all trips
                    # List with all start time | all end times
                    # All trips less than 1 minute | number of trips less than 1 minute
                    # All trips 1-3 minutes | number of trips 1-3 minutes
                    # All trips 3-6 minutes | number of trips 3-6 minutes
                    # All trips more than 6 minutes | number of trips more than 6 minutes

                    activity[month][day][key] = {"Day": 0, "Date": 0, "Group": "x", 
                    "Trip Durations": [], "Trip Durations (Seconds)": [], 
                    "Trip Amount": 0, "Trip Average Duration": 0, 
                    "Trip Average Duration (Seconds)": 0, "Rest Durations": [], 
                    "Rest Durations (Seconds)": [], "Rest Amount": 0, "Rest Average Duration": 0, 
                    "Rest Average Duration (Seconds)": 0, "First Detection": 0,
                    "First Detection (Seconds)": 0, "Last Detection": 0, 
                    "Last Detection (Seconds)": 0, "Trip Start and End Times": [], 
                    "Trip Start and End Times (Seconds)": [], "All Start Times": [],  
                    "All Start Times (Seconds)": [], "All End Times": [], "All End Times (Seconds)": [], 
                    "1 Minute or Less Trips": [], "1 Minute or Less Trips (Seconds)": [], "1 Minute Amount": 0, 
                    "1 Minute Average Duration": 0, "1 Minute Average Duration (Seconds)": 0, 
                    "1 Minute Rest Durations": [],  "1 Minute Rest Durations (Seconds)": [], 
                    "1 Minute Rest Amount": 0, "1 Minute Rest Average Duration": 0, 
                    "1 Minute Rest Average Duration (Seconds)": 0, "1-3 Minute Trips": [], 
                    "1-3 Minute Trips (Seconds)": [], "1-3 Minute Amount": 0, "1-3 Minute Average Duration": 0, 
                    "1-3 Minute Average Duration (Seconds)": 0, 
                    "1-3 Minute Rest Durations": [],  "1-3 Minute Rest Durations (Seconds)": [],
                    "1-3 Minute Rest Amount": 0, "1-3 Minute Rest Average Duration": 0,
                    "1-3 Minute Rest Average Duration (Seconds)": 0,
                    "3-6 Minute Trips": [], "3-6 Minute Trips (Seconds)": [],
                    "3-6 Minute Amount": 0, "3-6 Minute Average Duration": 0,
                    "3-6 Minute Average Duration (Seconds)": 0, 
                    "3-6 Minute Rest Durations": [],  "3-6 Minute Rest Durations (Seconds)": [],
                    "3-6 Minute Rest Amount": 0, "3-6 Minute Rest Average Duration": 0,
                    "3-6 Minute Rest Average Duration (Seconds)": 0, "6 Minute or More Trips": [],
                    "6 Minute or More Trips (Seconds)": [], "6 Minute Amount": 0, 
                    "6 Minute Average Duration": 0, "6 Minute Average Duration (Seconds)": 0,
                    "6 Minute Rest Durations": [],  "6 Minute Rest Durations (Seconds)": [], 
                    "6 Minute Rest Amount": 0, "6 Minute Rest Average Duration": 0,
                    "6 Minute Rest Average Duration (Seconds)": 0}

                    #activity[month][day][key]=[int(), "x", [], [], int(), int(), [], [], int(), [], int(), [], int(), [], int()]
                    if key in treatment_ids:
                        activity[month][day][key]["Group"]=treatment_ids[key]
                    else:
                        activity[month][day][key]["Group"]="No Group"
                    sames = []

                    prev_time=-1
                    add = 0.001
                    for j in range(i, ids):
                        if df["UID"][j] == key:
                            temp = df["Timestamp"][j]
                            #time_length=len(temp)
                            hour = int(temp[:2])
                            minute = int(temp[3:5])
                            second = int(float(temp[6:]))
                            # hour = int(temp[:time_length-6])
                            # minute = int(temp[time_length-5:time_length-3])
                            # second = int(float(temp[time_length-2:]))
                            # time = hour*3600+minute*60+second
                            time=hour+minute/60+second/3600
                            #print(time)
                            if time==prev_time:
                                time+=add
                                add+=0.00000001
                            else:
                                prev_time=time
                                add = 0.00000001

                            #print(temp, time, sep=" ")

                            sames.append([time, int(df["Ant. Number"][j])])
                       
                    first_key = True

                    sames.sort()     
                    #print(sames)
                    time_groups = []
                    reader_groups=[]
                    directions=[]

                    index = 0
                    secure_index=-1
                    activity[month][day][key]["First Detection"] = seconds_to_hours(sames[0][0])
                    activity[month][day][key]["First Detection (Seconds)"] = sames[0][0]
                    activity[month][day][key]["Last Detection"]=seconds_to_hours(sames[-1][0])
                    activity[month][day][key]["Last Detection (Seconds)"]=sames[-1][0]

                    while index<len(sames):
                        first_time=sames[index][0]
                        time_groups.append([])
                        reader_groups.append([])
                        directions.append("Unknown")
                        time_groups[-1].append(sames[index][0])
                        reader_groups[-1].append(sames[index][1])

                        index+=1
                        while index<len(sames):
                            if sames[index][0]-15/3600>first_time:
                                break
                            time_groups[-1].append(sames[index][0])
                            reader_groups[-1].append(sames[index][1])
                            index+=1

                        if len(reader_groups[-1])>=2:
                            if reader_groups[-1][0]==1 and reader_groups[-1][-1]==2:
                                directions[-1]="Arriving"
                            elif reader_groups[-1][0]==2 and reader_groups[-1][-1]==1:
                                directions[-1]="Departing"

                            if secure_index==-1 and (directions[-1]=="Arriving" or directions[-1]=="Departing"):
                                secure_index=len(directions)-1

                    time_groups=[[float(i) for i in block] for block in time_groups]
                    #print(time_groups)
                    #found a secure arrival or departure
                    if i!=-1:
                        possible_directions=[directions[secure_index], "x"]
                        if directions[secure_index]=="Arriving":
                            possible_directions[1]="Departing"
                        else:
                            possible_directions[1]="Arriving"

                        for i in range(len(directions)):
                            if directions[i]=="Unknown":
                                distance = abs(i-secure_index)%2
                                directions[i]=possible_directions[distance]

                    #if no secure arrival or departure, everything is unknown
                    #didn't happen a single time in the sample
                    
                    prev_time=-1
                    index=0
                    while index<len(time_groups)-1 and directions[index]=="Arriving":
                        index+=1
                    # latest "Departing", earliest "Arriving"            
                    while index<len(time_groups):
                        while index<len(time_groups)-1 and directions[index+1]=="Departing":
                            index+=1
                        # ends on a series of "Departing"
                        if index==len(time_groups)-1:
                            break


                        start_time=time_groups[index][0]
                        #print(start_time)
                        # first trip of the day for the bee
                        # if prev_time==-1:
                        #     activity[month][day][key]["First Detection"] = seconds_to_hours(start_time)
                        #     activity[month][day][key]["First Detection (Seconds)"] = start_time
                        # else:
                        #     activity[month][day][key]["Rest Average Duration (Seconds)"]+=start_time-prev_time
                        #     activity[month][day][key]["Rest Amount"]+=1
                        #     activity[month][day][key]["Rest Durations"].append(seconds_to_hours((start_time-prev_time)))
                        #     activity[month][day][key]["Rest Durations (Seconds)"].append((start_time-prev_time))

                        if prev_time!=-1:
                            activity[month][day][key]["Rest Average Duration (Seconds)"]+=start_time-prev_time
                            activity[month][day][key]["Rest Amount"]+=1
                            activity[month][day][key]["Rest Durations"].append(seconds_to_hours((start_time-prev_time)))
                            activity[month][day][key]["Rest Durations (Seconds)"].append((start_time-prev_time))

                        activity[month][day][key]["Trip Start and End Times"].append([seconds_to_hours(start_time)])
                        activity[month][day][key]["Trip Start and End Times (Seconds)"].append([start_time])
                        activity[month][day][key]["All Start Times"].append(seconds_to_hours(start_time))
                        activity[month][day][key]["All Start Times (Seconds)"].append(start_time)

                        index+=1
                        end_time = time_groups[index][0]
                        activity[month][day][key]["Trip Average Duration (Seconds)"] += end_time - \
                            start_time
                        activity[month][day][key]["Trip Amount"]+=1
                        activity[month][day][key]["Trip Durations"].append(seconds_to_hours(end_time-start_time))
                        activity[month][day][key]["Trip Durations (Seconds)"].append(end_time-start_time)

                        # activity[month][day][key]["Last Detection"]=seconds_to_hours(end_time)
                        # activity[month][day][key]["Last Detection (Seconds)"]=end_time
                        activity[month][day][key]["All End Times"].append(seconds_to_hours(end_time))
                        activity[month][day][key]["All End Times (Seconds)"].append(end_time)
                        activity[month][day][key]["Trip Start and End Times"][-1].append(seconds_to_hours(end_time))
                        activity[month][day][key]["Trip Start and End Times (Seconds)"][-1].append(end_time)
                        prev_time=end_time

                        if end_time-start_time<=60/3600:
                            if activity[month][day][key]["1 Minute Amount"]>=1:
                                rest_duration=start_time-activity[month][day][key]["1 Minute or Less Trips (Seconds)"][-1][1]
                                activity[month][day][key]["1 Minute Rest Durations"].append(seconds_to_hours(rest_duration))
                                activity[month][day][key]["1 Minute Rest Durations (Seconds)"].append(rest_duration)
                                activity[month][day][key]["1 Minute Rest Amount"]+=1
                                activity[month][day][key]["1 Minute Rest Average Duration (Seconds)"]+=rest_duration

                            activity[month][day][key]["1 Minute or Less Trips"].append([seconds_to_hours(start_time), seconds_to_hours(end_time)])
                            activity[month][day][key]["1 Minute or Less Trips (Seconds)"].append([start_time, end_time])
                            activity[month][day][key]["1 Minute Amount"]+=1
                            activity[month][day][key]["1 Minute Average Duration (Seconds)"]+=end_time-start_time
                        elif end_time-start_time <= 180/3600:
                            if activity[month][day][key]["1-3 Minute Amount"] >= 1:
                                rest_duration = start_time-activity[month][day][key][
                                    "1-3 Minute Trips (Seconds)"][-1][1]
                                activity[month][day][key]["1-3 Minute Rest Durations"].append(
                                    seconds_to_hours(rest_duration))
                                activity[month][day][key]["1-3 Minute Rest Durations (Seconds)"].append(
                                    rest_duration)
                                activity[month][day][key]["1-3 Minute Rest Amount"] += 1
                                activity[month][day][key]["1-3 Minute Rest Average Duration (Seconds)"] += rest_duration
                            activity[month][day][key]["1-3 Minute Trips"].append([seconds_to_hours(start_time), seconds_to_hours(end_time)])
                            activity[month][day][key]["1-3 Minute Trips (Seconds)"].append([start_time, end_time])
                            activity[month][day][key]["1-3 Minute Amount"]+=1
                            activity[month][day][key]["1-3 Minute Average Duration (Seconds)"]+=end_time-start_time
                        elif end_time-start_time <= 360/3600:
                            if activity[month][day][key]["3-6 Minute Amount"]>=1:
                                rest_duration=start_time-activity[month][day][key]["3-6 Minute Trips (Seconds)"][-1][1]
                                activity[month][day][key]["3-6 Minute Rest Durations"].append(seconds_to_hours(rest_duration))
                                activity[month][day][key]["3-6 Minute Rest Durations (Seconds)"].append(rest_duration)
                                activity[month][day][key]["3-6 Minute Rest Amount"]+=1
                                activity[month][day][key]["3-6 Minute Rest Average Duration (Seconds)"] += rest_duration
                            activity[month][day][key]["3-6 Minute Trips"].append([seconds_to_hours(start_time), seconds_to_hours(end_time)])
                            activity[month][day][key]["3-6 Minute Trips (Seconds)"].append([start_time, end_time])
                            activity[month][day][key]["3-6 Minute Amount"]+=1
                            activity[month][day][key]["3-6 Minute Average Duration (Seconds)"]+=end_time-start_time
                        else:
                            if activity[month][day][key]["6 Minute Amount"]>=1:
                                rest_duration=start_time-activity[month][day][key]["6 Minute or More Trips (Seconds)"][-1][1]
                                activity[month][day][key]["6 Minute Rest Durations"].append(seconds_to_hours(rest_duration))
                                activity[month][day][key]["6 Minute Rest Durations (Seconds)"].append(rest_duration)
                                activity[month][day][key]["6 Minute Rest Amount"]+=1
                                activity[month][day][key]["6 Minute Rest Average Duration (Seconds)"]+=rest_duration
                            activity[month][day][key]["6 Minute or More Trips"].append([seconds_to_hours(start_time), seconds_to_hours(end_time)])
                            activity[month][day][key]["6 Minute or More Trips (Seconds)"].append([start_time, end_time])
                            activity[month][day][key]["6 Minute Amount"]+=1
                            activity[month][day][key]["6 Minute Average Duration (Seconds)"]+=end_time-start_time
                            

                        while index<len(time_groups)-1 and directions[index+1]=="Arriving":
                            index+=1

        # for key in activity[month][day]:
        #     print(key, ": ", activity[month][day][key])

        # if file_count[month][day]==0:
                    activity[month][day][key]["Rest Average Duration"]=seconds_to_hours(activity[month][day][key]["Rest Average Duration (Seconds)"])

                    if activity[month][day][key]["Trip Amount"]>0:
                        activity[month][day][key]["Trip Average Duration (Seconds)"]/=activity[month][day][key]["Trip Amount"]
                        activity[month][day][key]["Trip Average Duration"]=seconds_to_hours(activity[month][day][key]["Trip Average Duration (Seconds)"])
                    if activity[month][day][key]["Rest Amount"]>0:
                        activity[month][day][key]["Rest Average Duration (Seconds)"] /= activity[month][day][key]["Rest Amount"]
                        activity[month][day][key]["Rest Average Duration"]=seconds_to_hours(activity[month][day][key]["Rest Average Duration (Seconds)"])

                    if activity[month][day][key]["1 Minute Amount"]>0:
                        activity[month][day][key]["1 Minute Average Duration (Seconds)"]/=activity[month][day][key]["1 Minute Amount"]
                        activity[month][day][key]["1 Minute Average Duration"]=seconds_to_hours(activity[month][day][key]["1 Minute Average Duration (Seconds)"])
                    if activity[month][day][key]["1 Minute Rest Amount"]>0:
                        activity[month][day][key]["1 Minute Rest Average Duration (Seconds)"] /= activity[month][day][key]["1 Minute Rest Amount"]
                        activity[month][day][key]["1 Minute Rest Average Duration"] = seconds_to_hours(activity[month][day][key]["1 Minute Rest Average Duration (Seconds)"])

                    if activity[month][day][key]["1-3 Minute Amount"]>0:
                        activity[month][day][key]["1-3 Minute Average Duration (Seconds)"]/=activity[month][day][key]["1-3 Minute Amount"]
                        activity[month][day][key]["1-3 Minute Average Duration"]=seconds_to_hours(activity[month][day][key]["1-3 Minute Average Duration (Seconds)"])
                    if activity[month][day][key]["1-3 Minute Rest Amount"]>0:
                        activity[month][day][key]["1-3 Minute Rest Average Duration (Seconds)"] /= activity[
                            month][day][key]["1-3 Minute Rest Amount"]
                        activity[month][day][key]["1-3 Minute Rest Average Duration"] = seconds_to_hours(
                            activity[month][day][key]["1-3 Minute Rest Average Duration (Seconds)"])

                    if activity[month][day][key]["3-6 Minute Amount"]>0:
                        activity[month][day][key]["3-6 Minute Average Duration (Seconds)"]/=activity[month][day][key]["3-6 Minute Amount"]
                        activity[month][day][key]["3-6 Minute Average Duration"]=seconds_to_hours(activity[month][day][key]["3-6 Minute Average Duration (Seconds)"])
                    if activity[month][day][key]["3-6 Minute Rest Amount"]>0:
                        activity[month][day][key]["3-6 Minute Rest Average Duration (Seconds)"] /= activity[month][day][key]["3-6 Minute Rest Amount"]
                        activity[month][day][key]["3-6 Minute Rest Average Duration"] = seconds_to_hours(
                            activity[month][day][key]["3-6 Minute Rest Average Duration (Seconds)"])

                    if activity[month][day][key]["6 Minute Amount"]>0:
                        activity[month][day][key]["6 Minute Average Duration (Seconds)"]/=activity[month][day][key]["6 Minute Amount"]
                        activity[month][day][key]["6 Minute Average Duration"] = seconds_to_hours(activity[month][day][key]["6 Minute Average Duration (Seconds)"])
                    if activity[month][day][key]["6 Minute Rest Amount"]>0:
                        activity[month][day][key]["6 Minute Rest Average Duration (Seconds)"] /= activity[month][day][key]["6 Minute Rest Amount"]
                        activity[month][day][key]["6 Minute Rest Average Duration"] = seconds_to_hours(activity[month][day][key]["6 Minute Rest Average Duration (Seconds)"])

        # month_df = pd.DataFrame.from_dict(activity[month][day], orient="index")
        #print(month_df.columns)
        # month_df.to_csv(directory+'/'+processed_folder+'/type3_'+str(month+1)+'_'+str(day+1)+".csv")
        
# months=["January","February","March","April","May","June","July",
#             "August","September","October","November","December"]
months_num=[31,28,31,30,31,30,31,31,30,31,30,31]
# for month in range (len(activity)):
#     print(months[month])
#     for day in range (months_num[month]):
#         print("Day: ", day)
#         print(activity[month][day])

final_df=[]
day=1
for i in range(start_day, months_num[start_month]):
    for key in activity[start_month][i]:
        activity[start_month][i][key]["Day"]=day
        activity[start_month][i][key]["Date"]=start_month+i/100
        
    final_df.append(pd.DataFrame.from_dict(activity[start_month][i], orient="index"))
    day+=1

for i in range(start_month+1, end_month):
    for j in range(months_num[i]):
        for key in activity[i][j]:
            activity[i][j][key]["Day"]=day
            activity[i][j][key]["Date"]=i+j/100

        final_df.append(pd.DataFrame.from_dict(activity[i][j], orient="index"))
        day+=1

for i in range(end_day+1):
    for key in activity[end_month][i]:
        activity[end_month][i][key]["Day"]=day
        activity[end_month][i][key]["Date"]=end_month+i/100
        
    final_df.append(pd.DataFrame.from_dict(
        activity[end_month][i], orient="index"))
    day+=1

# final_df

column_headers = ["Day", "Date", "Group", "Trip Durations", "Trip Durations (Seconds)",
                  "Trip Amount", "Trip Average Duration",
                  "Trip Average Duration (Seconds)", "Rest Durations",
                  "Rest Durations (Seconds)", "Rest Amount", "Rest Average Duration",
                  "Rest Average Duration (Seconds)", "First Detection",
                  "First Detection (Seconds)", "Last Detection",
                  "Last Detection (Seconds)", "Trip Start and End Times",
                  "Trip Start and End Times (Seconds)", "All Start Times",
                  "All Start Times (Seconds)", "All End Times", "All End Times (Seconds)",
                  "1 Minute or Less Trips", "1 Minute or Less Trips (Seconds)", "1 Minute Amount",
                  "1 Minute Average Duration", "1 Minute Average Duration (Seconds)",
                  "1 Minute Rest Durations",  "1 Minute Rest Durations (Seconds)",
                  "1 Minute Rest Amount", "1 Minute Rest Average Duration",
                  "1 Minute Rest Average Duration (Seconds)", "1-3 Minute Trips",
                  "1-3 Minute Trips (Seconds)", "1-3 Minute Amount", "1-3 Minute Average Duration",
                  "1-3 Minute Average Duration (Seconds)",
                  "1-3 Minute Rest Durations",  "1-3 Minute Rest Durations (Seconds)",
                  "1-3 Minute Rest Amount", "1-3 Minute Rest Average Duration",
                  "1-3 Minute Rest Average Duration (Seconds)",
                  "3-6 Minute Trips", "3-6 Minute Trips (Seconds)",
                  "3-6 Minute Amount", "3-6 Minute Average Duration",
                  "3-6 Minute Average Duration (Seconds)", "6 Minute or More Trips",
                  "3-6 Minute Rest Durations",  "3-6 Minute Rest Durations (Seconds)",
                  "3-6 Minute Rest Amount", "3-6 Minute Rest Average Duration",
                  "3-6 Minute Rest Average Duration (Seconds)",
                  "6 Minute or More Trips (Seconds)", "6 Minute Amount",
                  "6 Minute Average Duration", "6 Minute Average Duration (Seconds)",
                  "6 Minute Rest Durations",  "6 Minute Rest Durations (Seconds)",
                  "6 Minute Rest Amount", "6 Minute Rest Average Duration", 
                  "6 Minute Rest Average Duration (Seconds)"]
# print(start_month, start_day, end_month, end_day, day, sep=" ")

final_df_csv=pd.concat(final_df)

final_df_csv.to_csv(directory+'/'+processed_folder+'/processed.csv', header=column_headers)

# List with durations of trips in sequential order |
                    # List with durations of time between trips in sequential order |
                    # Start time of first trip | end time of last trip
                    # List with start and end times of all trips
